In [1]:
import IPython.display
import pandas
import np_session

In [8]:
def summary_df(session):
    names = session.D1.names + session.D2.names
    globs = session.D1.globs + session.D2.globs
    paths = session.D1.paths + session.D2.paths
    types = session.D1.types + session.D2.types

    print(session.npexp_path)
    return pandas.DataFrame(({'glob': glob, 'exists': path.name if path else None} for glob, path in zip(globs, paths)), index=names)

In [19]:
all_exists: dict[str, bool] = dict()
all_dfs: dict[str, pandas.DataFrame] = dict()
missing: dict[str, tuple[str, ...]] = dict()

for session in np_session.sessions('DR'):
    df = summary_df(session)
    all_exists[session.folder] = all(df['exists'].values)
    all_dfs[session.folder] = df
    # missing[session.folder] = session.get_missing_files()

KeyboardInterrupt: 

In [18]:
all_exists[np_session.Session('1178693650_608671_20220520')]

False

In [16]:
len([_ for _ in all_exists.values() if _ == True])
len(all_exists.values())

124